## This notebook is only used for the Applied Data Science Capstone on Coursera.

### Capstone Project Notebook

In [1]:
import pandas as pd 
import numpy as np

In [2]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


### Segmenting and Clustering Neighborhoods in Toronto

#### Part A of Week 3

Install dependencies for BeautifulSoup

In [3]:
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
import requests

Pull data from wikipedia using BeautifulSoup

In [4]:
# Use the Notebook to build the code to scrape the following Wikipedia page, 
# https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order 
# to obtain the data that is in the table of postal codes and to transform 
# the data into a pandas dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

Search data for specified string associated with data of interest (Table)

In [5]:
table = soup.find("table", attrs={'class': 'wikitable'})
tds = table.tbody.find_all('tr')

#Get the header
headings = []
for td in tds[0].find_all('th'):
    headings.append(td.contents[0].strip()) 

headings

['Postal Code', 'Borough', 'Neighbourhood']

Loop through data to obtain rows of our dataframe. Elimination un-needed items and replacing some. 

In [6]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
data = []

for tr in tds[1:]:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if not ('Not assigned' in row[1]):   #Ignore cells with a borough that is Not assigned.
        if 'Not assigned' in row[2]:
            row[2] = row[1]                 #Replace Neighbourhood for borough if 'Not assigned'.  
        data.append(row)
    
#create dataframe   
df = pd.DataFrame(data, columns=headings)

#combime Neighbourhoods with same postal code
df = df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
#print the numbre of rows of your dataframe
print("Shape: ", df.shape)

Shape:  (103, 3)


#### Part B of Week 3

Attempting to use geocoder

In [8]:
!pip install geocoder
import geocoder # import geocoder

In [9]:
'''
# initialize your variable to None
postal_codes = df['Postal Code'].tolist()
geo_coord = []
# loop until you get the coordinates
for postal_code in postal_codes:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    geo_coord.append([postal_code, lat_lng_coords[0], lat_lng_coords[1]])
'''

"\n# initialize your variable to None\npostal_codes = df['Postal Code'].tolist()\ngeo_coord = []\n# loop until you get the coordinates\nfor postal_code in postal_codes:\n    lat_lng_coords = None\n    while(lat_lng_coords is None):\n        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n        lat_lng_coords = g.latlng\n    geo_coord.append([postal_code, lat_lng_coords[0], lat_lng_coords[1]])\n"

Reading csv to save time

In [10]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_toronto = pd.merge(df, df_geo, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


#### Part C of Week 3

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
from geopy.geocoders import Nominatim 
import folium

Create a map of New York with neighborhoods superimposed on top.

In [23]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
    df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
        
map_toronto

Define Foursquare Credentials and Version

In [40]:
CLIENT_ID = 'N3Q1ZQVOL4JF1E1WPTX1UEWRJSLDJWBXXF4BGQQARVITKAZ2' # your Foursquare ID
CLIENT_SECRET = 'X1EFUL5UOS5XESJT3NBGKZXFZRAFDPKCI1XSFZRULALAJJRW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N3Q1ZQVOL4JF1E1WPTX1UEWRJSLDJWBXXF4BGQQARVITKAZ2
CLIENT_SECRET:X1EFUL5UOS5XESJT3NBGKZXFZRAFDPKCI1XSFZRULALAJJRW


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

function to loop through boroughs and get data from foursquare

In [42]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

Run above function for toronto data

In [43]:
print(toronto_venues.shape)
toronto_venues.head()

(2136, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4
Woodbine Heights,7,7,7,7,7,7


check how many venues were returned for each neighborhood

unique categories can be curated from all the returned venues

In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.
